<a href="https://colab.research.google.com/github/captain7oxic/Pytorch_practice/blob/main/DeeLearning_part_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we will try to calculate loss as well as implement back-propogation of a neural network using pytorch.

In [1]:
#importing the data
from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import datasets, transforms

In [4]:
#define a Transform to Normalize the data
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5),(0.5)),])

In [6]:
#collecting all the data and running the normalization on entire data
trainset = datasets.MNIST('~/.pytorch/MNIST_data/',download=True,train=True,transform = transform)

100%|██████████| 9912422/9912422 [00:00<00:00, 105060005.72it/s]


Extracting /root/.pytorch/MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 19270711.71it/s]

Extracting /root/.pytorch/MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 25964744.30it/s]

Extracting /root/.pytorch/MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 8906278.06it/s]


Extracting /root/.pytorch/MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw



In [8]:
#defining the batchsize and shuffling the data in the loader
loader  = torch.utils.data.DataLoader(trainset,batch_size = 64,shuffle = True)

In [11]:
#creating a feed-forward Neural Network

Net = nn.Sequential(
    nn.Linear(784,128),
    nn.ReLU(),
    nn.Linear(128,64),
    nn.ReLU(),
    nn.Linear(64,10)
)

In [12]:
#define our loss
loss = nn.CrossEntropyLoss()

There are many loss functions
(1)**Regression Loss Functions**
Mean Squared Error Loss

Mean Squared Logarithmic Error Loss

Mean Absolute Error Loss *italicized text*

(2)**Binary Classification Loss Functions**

Binary Cross-Entropy

Hinge Loss

Squared Hinge Loss *italicized text*

(3)**Multi-Class Classification Loss Functions**

Multi-Class Cross-Entropy Loss

Sparse Multiclass Cross-Entropy Loss

Kullback Leibler Divergence Loss *italicized text*

In [14]:
#iterate over the data
data_iter = iter(loader)

images,labels = next(data_iter)

In [15]:
#flatten the images
images = images.view(images.shape[0],-1)

In [17]:
# Forward pass, get our logits
logits = Net(images)

In [19]:
total_loss=loss(logits,labels)
print(total_loss)

tensor(2.3158, grad_fn=<NllLossBackward0>)


Exercise: Build a model that returns the log-softmax as the output and calculate the loss using the negative log likelihood loss. Note that for nn.LogSoftmax and F.log_softmax you'll need to set the dim keyword argument appropriately. dim=0 calculates softmax across the rows, so each column sums to 1, while dim=1 calculates across the columns so each row sums to 1. Think about what you want the output to be and choose dim appropriately.

In [22]:
from torch.nn.modules.activation import LogSoftmax
# TODO: Build a feed-forward network
model = nn.Sequential(
    nn.Linear(784,128),
    nn.ReLU(),
    nn.Linear(128,64),
    nn.LogSoftmax(dim=1),
    nn.Linear(64,10)
)

# TODO: Define the loss
criterion = nn.NLLLoss()

### Run this to check your work
# Get our data
dataiter = iter(loader)

images, labels = next(dataiter)

# Flatten images
images = images.view(images.shape[0], -1)

# Forward pass, get our logits
logits = model(images)
# Calculate the loss with the logits and the labels
loss = criterion(logits, labels)

print(loss)

tensor(-0.5001, grad_fn=<NllLossBackward0>)


clearly the loss is reduced in this example , now we will try to use AutoGrad . Autograd typically stores the trace of calculations happening over the tensor , which helps to make the chained derivatives calculations faster.

In [23]:
x = torch.randn(2,2, requires_grad=True)
print(x)

tensor([[-0.7793,  1.2771],
        [ 0.4286,  0.4775]], requires_grad=True)


In [24]:
y = x**2
print(y)

tensor([[0.6073, 1.6311],
        [0.1837, 0.2280]], grad_fn=<PowBackward0>)


In [25]:
## grad_fn shows the function that generated this variable
print(y.grad_fn)

In [26]:
z= y.mean()

In [27]:
z.backward()
print(x.grad)
print(x/2)

tensor([[-0.3896,  0.6386],
        [ 0.2143,  0.2388]])
tensor([[-0.3896,  0.6386],
        [ 0.2143,  0.2388]], grad_fn=<DivBackward0>)


In [28]:
#now lets use Autograd and Loss functions together

In [31]:
model = nn.Sequential(
    nn.Linear(784,128),
    nn.ReLU(),
    nn.Linear(128,64),
    nn.ReLU(),
    nn.Linear(64,10),
    nn.LogSoftmax(dim=1)
)

criterion = nn.NLLLoss()

dataiter = iter(loader)
images, labels = next(dataiter)
images = images.view(images.shape[0], -1)

logits = model(images)
loss = criterion(logits,labels)

In [32]:
loss

tensor(2.3098, grad_fn=<NllLossBackward0>)

In [33]:
print('Before backward pass: \n', model[0].weight.grad)

loss.backward()

print('After backward pass: \n', model[0].weight.grad)

Before backward pass: 
 None
After backward pass: 
 tensor([[-0.0008, -0.0008, -0.0008,  ..., -0.0008, -0.0008, -0.0008],
        [ 0.0004,  0.0004,  0.0004,  ...,  0.0004,  0.0004,  0.0004],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0049,  0.0049,  0.0049,  ...,  0.0049,  0.0049,  0.0049],
        [ 0.0010,  0.0010,  0.0010,  ...,  0.0010,  0.0010,  0.0010],
        [ 0.0051,  0.0051,  0.0051,  ...,  0.0051,  0.0051,  0.0051]])


For The garnish we have the Optim package of the pytorch library to enable Gradient descent calculations on the network.

In [34]:
from torch import optim

In [35]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

torch.optim.sgd.SGD